In [1]:
import os
import sys
sys.path.append(os.path.abspath(r"C:\Users\Site Oracle\Live-Sports-Trader\src"))
import domain
import repository 
import pandas as pd
from dotenv import load_dotenv
from py_clob_client.constants import POLYGON
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds

# Add the directory containing polymarket_functions.py to the system path
sys.path.append(os.path.abspath(r"C:\Users\Site Oracle\polymarket_sniper\Production"))
sys.path.append(os.path.abspath(r"C:\Users\Site Oracle\Live-Sports-Trader\src"))

# Now you can import the module
import polymarket_functions
import repository
from domain import Odds, Game, Team
import service

In [2]:
client = polymarket_functions.authenticate()

In [3]:
res = repository.get_polymarket_team_odds_today("MLB game")

In [4]:
res.iloc[0]['tokens']

"[{'token_id': '100847193076407127628422261720419835818363338235872492949159441281075764967389', 'outcome': 'Cubs', 'price': 0.485, 'winner': False}, {'token_id': '53356564475086714840382487736621324083414936999788684814497633438156373677611', 'outcome': 'Phillies', 'price': 0.515, 'winner': False}]"

In [ ]:

games = []
import json 

for _, row in res.iterrows():
    game_name = row['question']
    game = Game(game_name)

    # Parse team names from the question (assumes format "Team1 vs. Team2")
    team_names = [name.strip() for name in game_name.split(' vs. ')]

    # Extract tokens data - handle both string and list formats
    tokens_data = row['tokens']
    if isinstance(tokens_data, str):
        # More robust parsing for Python-like string representation
        try:
            # Replace Python booleans and None with JSON equivalents
            json_str = (tokens_data
                        .replace("'", '"')
                        .replace('False', 'false')
                        .replace('True', 'true')
                        .replace('None', 'null'))
            tokens_data = json.loads(json_str)
        except json.JSONDecodeError:
            # If JSON parsing fails, try using ast.literal_eval as fallback
            import ast
            tokens_data = ast.literal_eval(tokens_data)

    # Create Team objects
    for i, token_data in enumerate(tokens_data):
        if i < len(team_names):
            team_name = team_names[i]
            token_id = token_data['token_id']
            
            team = Team(name=team_name, token_id=token_id)
            game.teams.append(team)

    games.append(game)

In [6]:
for game in games:
    print(game.name)
    for team in game.teams:
        print(team.name)
        print(team.token_id)
        print(team.prediction_market_odds)
        print(team.true_odds)

Cubs vs. Phillies
Cubs
100847193076407127628422261720419835818363338235872492949159441281075764967389
None
None
Phillies
53356564475086714840382487736621324083414936999788684814497633438156373677611
None
None
Blue Jays vs. Cardinals
Blue Jays
105654570225018626452809322051286489211889548453414775688656851531200140985882
None
None
Cardinals
16339224110951785496956968399073515764710855920394301901142267391182143120843
None
None
White Sox vs. Astros
White Sox
7845273941001892808258397164688631344039837594188526418429655642528201308088
None
None
Astros
8124785495960885783755513431404366229228879663459080054643686285203963598357
None
None
Mariners vs. Diamondbacks
Mariners
2386908882778522624005309579591982349264240318277251898318232861412636248314
None
None
Diamondbacks
67976380644485985935636441035817510422473859638401786442955240176885455291924
None
None
Tigers vs. Orioles
Tigers
85471906673049084184462355804346818496211949248881924587897420209115006926115
None
None
Orioles
6242467614899

In [7]:
driver = repository.get_driver_fanduel_live_odds_page_existing_chrome()

live_odds = repository.parse_live_odds(driver=driver)
print(live_odds)

{'Toronto Blue Jays': '-6000', 'St. Louis Cardinals': '+1400', 'Seattle Mariners': '-225', 'Arizona Diamondbacks': '+172', 'Athletics': '-230', 'Los Angeles Angels': '+176', 'Los Angeles Dodgers': '+132', 'San Diego Padres': '-170'}


In [8]:

for game in games:

    for team in game.teams:
        team.prediction_market_odds = Odds(probability=float(polymarket_functions.get_current_price(client, team.token_id)[team.token_id]['BUY']))
        for key, value in live_odds.items():
            if team.name.lower() in ' '.join(key.split()[1:]).lower():
                team.true_odds = Odds(american_odds=live_odds[key])

    no_vig_dict = game.get_no_vig_odds(odds_type='true_odds')

    for team in game.teams:
        if no_vig_dict is None:
            continue
        team.no_vig_true_odds = no_vig_dict['team1_odds'] if team.name.lower() == no_vig_dict['team1_name'].lower() else no_vig_dict['team2_odds']


total probability: 1.0502732240437158
no vig team1 prob: 0.9365244536940687
no vig team2 prob: 0.06347554630593132
total probability: 1.0599547511312217
no vig team1 prob: 0.6531483457844184
no vig team2 prob: 0.3468516542155817
total probability: 1.0606641123882503
no vig team1 prob: 0.4063816977724262
no vig team2 prob: 0.5936183022275737


In [9]:
for game in games:
    for team in game.teams:
        if team.true_odds is not None:
            print(f"\nTeam: {team.name}")
            print(f"true odds: {team.true_odds}")
            print(f"no vig true odds: {team.no_vig_true_odds}")
            print(team.get_difference_between_desired_expected_value_and_market_bid(desired_expected_value=0.25))


Team: Blue Jays
true odds: -6000 - 0.9836
no vig true odds: -1475 - 0.9365
Desired limit price: 0.7492195629552549
Market bid: 0.96
-0.21078043704474503

Team: Cardinals
true odds: +1400 - 0.0667
no vig true odds: +1475 - 0.0635
Desired limit price: 0.05078043704474505
Market bid: 0.026
0.024780437044745054

Team: Mariners
true odds: -225 - 0.6923
no vig true odds: -188 - 0.6531
Desired limit price: 0.5225186766275347
Market bid: 0.63
-0.1074813233724653

Team: Diamondbacks
true odds: +172 - 0.3676
no vig true odds: +188 - 0.3469
Desired limit price: 0.27748132337246534
Market bid: 0.35
-0.07251867662753464

Team: Dodgers
true odds: +132 - 0.4310
no vig true odds: +146 - 0.4064
Desired limit price: 0.32510535821794095
Market bid: 0.39
-0.06489464178205906

Team: Padres
true odds: -170 - 0.6296
no vig true odds: -146 - 0.5936
Desired limit price: 0.474894641782059
Market bid: 0.59
-0.11510535821794099

Team: Angels
true odds: +176 - 0.3623
no vig true odds: None


AttributeError: 'NoneType' object has no attribute 'probability'